In [2]:
import pandas as pd
import numpy as np
from ast import literal_eval

In [3]:
df = pd.read_csv('data/movies_metadata.csv.zip')

/tmp/ipykernel_197633/2680075255.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/movies_metadata.csv.zip')


In [4]:
df.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [5]:
df.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


$$
Weighted Rating (WR) = (\frac{v}{v+m} *R) + (\frac{m}{v+m} * C)
$$

In [6]:
# min no of votes
m = df['vote_count'].quantile(0.80)

In [7]:
q_movies = df[(df['runtime'] >= 45) & (df['runtime'] <= 300)]

q_movies = q_movies[q_movies['vote_count'] >= m]

q_movies.shape

(8963, 24)

In [8]:
C = df['vote_average'].mean()
C

np.float64(5.618207215134185)

In [9]:
def weighted_rating(x, m=m, C=C):
    v = x['vote_count']
    R = x['vote_average']
    return (v/(v+m)*R)+(m/(m+v)*C)

In [10]:
q_movies['score'] = q_movies.apply(weighted_rating, axis=1)

In [11]:
df = df[['title', 'genres', 'release_date', 'runtime', 'vote_average', 'vote_count']]
df.head()

,title,genres,release_date,runtime,vote_average,vote_count
0,Toy Story,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",1995-10-30,81.0,7.7,5415.0
1,Jumanji,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",1995-12-15,104.0,6.9,2413.0
2,Grumpier Old Men,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",1995-12-22,101.0,6.5,92.0
3,Waiting to Exhale,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",1995-12-22,127.0,6.1,34.0
4,Father of the Bride Part II,"[{'id': 35, 'name': 'Comedy'}]",1995-02-10,106.0,5.7,173.0


In [12]:
df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce')

In [13]:
df['year'] = df['release_date'].apply(lambda x: str(x).split('-')[0] if x != np.nan else np.nan)

In [14]:
df

,title,genres,release_date,runtime,vote_average,vote_count,year
0,Toy Story,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",1995-10-30,81.0,7.7,5415.0,1995
1,Jumanji,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",1995-12-15,104.0,6.9,2413.0,1995
2,Grumpier Old Men,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",1995-12-22,101.0,6.5,92.0,1995
3,Waiting to Exhale,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",1995-12-22,127.0,6.1,34.0,1995
4,Father of the Bride Part II,"[{'id': 35, 'name': 'Comedy'}]",1995-02-10,106.0,5.7,173.0,1995
...,...,...,...,...,...,...,...
45461,Subdue,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",NaT,90.0,4.0,1.0,NaT
45462,Century of Birthing,"[{'id': 18, 'name': 'Drama'}]",2011-11-17,360.0,9.0,3.0,2011
45463,Betrayal,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",2003-08-01,90.0,3.8,6.0,2003
45464,Satan Triumphant,[],1917-10-21,87.0,0.0,0.0,1917


In [15]:
def convert_int(x):
    try:
        return int(x)
    except:
        return 0

In [16]:
df['year'] = df['year'].apply(convert_int)

In [17]:
df = df.drop('release_date', axis=1)
df.head()

,title,genres,runtime,vote_average,vote_count,year
0,Toy Story,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",81.0,7.7,5415.0,1995
1,Jumanji,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",104.0,6.9,2413.0,1995
2,Grumpier Old Men,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",101.0,6.5,92.0,1995
3,Waiting to Exhale,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",127.0,6.1,34.0,1995
4,Father of the Bride Part II,"[{'id': 35, 'name': 'Comedy'}]",106.0,5.7,173.0,1995


In [18]:
df.iloc[0]['genres']

"[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]"

In [19]:
df['genres'] = df['genres'].fillna('[]')
df['genres'] = df['genres'].apply(literal_eval)

In [20]:
df['genres'] = df['genres'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

In [21]:
df.head()

,title,genres,runtime,vote_average,vote_count,year
0,Toy Story,"[Animation, Comedy, Family]",81.0,7.7,5415.0,1995
1,Jumanji,"[Adventure, Fantasy, Family]",104.0,6.9,2413.0,1995
2,Grumpier Old Men,"[Romance, Comedy]",101.0,6.5,92.0,1995
3,Waiting to Exhale,"[Comedy, Drama, Romance]",127.0,6.1,34.0,1995
4,Father of the Bride Part II,[Comedy],106.0,5.7,173.0,1995


In [22]:
df = df.explode('genres').reset_index(drop=True)

In [23]:
df

,title,genres,runtime,vote_average,vote_count,year
0,Toy Story,Animation,81.0,7.7,5415.0,1995
1,Toy Story,Comedy,81.0,7.7,5415.0,1995
2,Toy Story,Family,81.0,7.7,5415.0,1995
3,Jumanji,Adventure,104.0,6.9,2413.0,1995
4,Jumanji,Fantasy,104.0,6.9,2413.0,1995
...,...,...,...,...,...,...
93543,Betrayal,Action,90.0,3.8,6.0,2003
93544,Betrayal,Drama,90.0,3.8,6.0,2003
93545,Betrayal,Thriller,90.0,3.8,6.0,2003
93546,Satan Triumphant,NaN,87.0,0.0,0.0,1917


In [ ]:
def build_chart(df, percentile=0.80):
    genre = input('Input preferred genre: ')
    low_time = int(input('Input shortest duration: '))
    high_time = int(input('Input longest duration: '))
    low_year = int(input('Input earliest year: '))
    high_year = int(input('Input latest year: '))

    movies = df.copy()
    # print(movies.shape)
    movies = movies[(movies['genres'] == genre) & (movies['runtime'] >= low_time) &
                    (movies['runtime'] <= high_time) &
                    (movies['year'] >= low_year) & (movies['year']<= high_year)]
    
    C = movies['vote_average'].mean()
    m = movies['vote_count'].quantile(percentile)

    q_movies = movies.copy().loc[movies['vote_count'] >= m]
    q_movies['score'] = q_movies.apply(lambda x: (x['vote_count']/(x['vote_count']+m)
                                                  * x['vote_average']) + 
                                                  (m/(m+x['vote_count'])*C), axis=1)
    
    q_movies = q_movies.sort_values('score', ascending=False)
    return q_movies

In [25]:
build_chart(df).head()

(93548, 6)


,title,genres,runtime,vote_average,vote_count,year,score
13301,Spirited Away,Animation,125.0,8.3,3968.0,2001,8.200247
23304,Howl's Moving Castle,Animation,119.0,8.2,2049.0,2004,8.023473
6972,Princess Mononoke,Animation,134.0,8.2,2041.0,1997,8.022841
877,The Lion King,Animation,89.0,8.0,5520.0,1994,7.937481
13476,Grave of the Fireflies,Animation,89.0,8.2,974.0,1988,7.860862
